In [ ]:
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [ ]:
image_dir = "path_to_keograms"
images = []
for file in os.listdir(image_dir):
    img = cv2.imread(os.path.join(image_dir, file), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        images.append(img.flatten())  # Flatten the image into a vector


In [ ]:
# Convert to a NumPy array
images = np.array(images)

# Standardize the data
scaler = StandardScaler()
images_scaled = scaler.fit_transform(images)




In [ ]:
# Apply PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
images_pca = pca.fit_transform(images_scaled)

# Apply K-Means Clustering
kmeans = KMeans(n_clusters=5)  # Choose the number of clusters
labels = kmeans.fit_predict(images_pca)